In [5]:
#Q1-1) 아래에 주어진 주석을 기반으로 하여 코딩을 해주세요.

import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pylab as plt
import random


# 파라미터 설정 (learning rate, training epochs, batch_size)
learning_rate = 0.1
training_epochs = 15
batch_size = 100


#train과 test set으로 나누어 MNIST data 불러오기

mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)


#dataset loader에 train과 test할당하기(batch size, shuffle, drop_last 잘 설정할 것!)

train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Layer 쌓기 (조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), ReLU 함수 사용, Batch normalization하기)
# 각 Layer의 Hidden node 수 : 1st Layer (784,100), 2nd Layer(100,100),3rd Layer(100,10)

linear1 = torch.nn.Linear(784, 100, bias=True)
linear2 = torch.nn.Linear(100, 100, bias=True)
linear3 = torch.nn.Linear(100, 10, bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=0.3)

bn1 = torch.nn.BatchNorm1d(100)
bn2 = torch.nn.BatchNorm1d(100)

#xavier initialization을 이용하여 각 layer의 weight 초기화

torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)


# torch.nn.Sequential을 이용하여 model 정의하기(쌓는 순서: linear-Batch Normalization Layer - ReLU- DropOut)

model = torch.nn.Sequential(linear1, bn1, relu, dropout,
                            linear2, bn2, relu, dropout,
                            linear3).to(device)

In [15]:
# Loss Function 정의하기 (CrossEntropy를 사용할 것!)

criterion = torch.nn.CrossEntropyLoss().to(device)

#optimizer 정의하기 (Adam optimizer를 사용할 것!)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#cost 계산을 위한 변수 설정
train_total_batch = len(train_loader)

#Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것) 
model.train()
for epoch in range(training_epochs):
    avg_cost = 0
    
    #train dataset을 불러오고(X,Y 불러오기), back propagation과 optimizer를 사용하여 loss를 최적화하는 코드
    for X, Y in train_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        bn_loss = criterion(hypothesis, Y)
        bn_loss.backward()
        optimizer.step()

        avg_cost += bn_loss / train_total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.500336289
Epoch: 0002 cost = 0.364107549
Epoch: 0003 cost = 0.318127483
Epoch: 0004 cost = 0.297909141
Epoch: 0005 cost = 0.292953074
Epoch: 0006 cost = 0.286360711
Epoch: 0007 cost = 0.267688602
Epoch: 0008 cost = 0.272522599
Epoch: 0009 cost = 0.251771331
Epoch: 0010 cost = 0.255685508
Epoch: 0011 cost = 0.246699750
Epoch: 0012 cost = 0.239352673
Epoch: 0013 cost = 0.246473223
Epoch: 0014 cost = 0.241315573
Epoch: 0015 cost = 0.231282264
Learning finished


In [17]:
#test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것)
#X_test 불러올 때 view를 사용하여 차원 변환할 것/ Y_test를 불러올때 labels사용
#accuracy의 초기 값 설정(0으로) 꼭 할 것

with torch.no_grad():
    model.eval()
    
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    bn_acc = correct_prediction.float().mean()
    print('Accuracy:', bn_acc.item())
    
    ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    
    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9312999844551086
Label:  7
Prediction:  7


Q1-2) 지금까지는 Layer의 수를 바꾸거나, Batch Normalization Layer를 추가하는 등 Layer에만 변화를 주며 모델의 성능을 향상 시켰습니다. **
이번 문제에서는 위에서 만든 모델에서 있던 Layer 들의 Hidden node 수를 증가 또는 감소 (ex: 200, 300, 50...) 시켰을 때, train set에서의 cost와 test set에서 Accuracy가 기존 결과와 비교하였을 때 어떻게 달라졌는지 비교해주시면 됩니다.

**Hidden node 수 변화 시 주의해야 할 점 **

각 Layer의 앞 뒤 node 숫자 일치시키기

Batch Normalizaion Layer의 node 숫자와 일치시키기

In [22]:
# 200개라면?
acc_list = bn_acc.tolist()

# Layer 쌓기 (조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), ReLU 함수 사용, Batch normalization하기)
# 각 Layer의 Hidden node 수 : 1st Layer (784,200), 2nd Layer(200,200),3rd Layer(200,10)

linear4 = torch.nn.Linear(784, 200, bias=True)
linear5 = torch.nn.Linear(200, 200, bias=True)
linear6 = torch.nn.Linear(200, 10, bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=0.3)

bn3 = torch.nn.BatchNorm1d(200)
bn4 = torch.nn.BatchNorm1d(200)

#xavier initialization을 이용하여 각 layer의 weight 초기화

torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)
torch.nn.init.xavier_uniform_(linear6.weight)


# torch.nn.Sequential을 이용하여 model 정의하기(쌓는 순서: linear-Batch Normalization Layer - ReLU- DropOut)

model2 = torch.nn.Sequential(linear4, bn3, relu, dropout,
                             linear5, bn4, relu, dropout,
                             linear6).to(device)

In [23]:
# Loss Function 정의하기 (CrossEntropy를 사용할 것!)

criterion = torch.nn.CrossEntropyLoss().to(device)

#optimizer 정의하기 (Adam optimizer를 사용할 것!)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#cost 계산을 위한 변수 설정
train_total_batch = len(train_loader)

#Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것) 
model2.train()
for epoch in range(training_epochs):
    avg_cost = 0
    
    #train dataset을 불러오고(X,Y 불러오기), back propagation과 optimizer를 사용하여 loss를 최적화하는 코드
    for X, Y in train_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model2(X)
        bn_loss = criterion(hypothesis, Y)
        bn_loss.backward()
        optimizer.step()

        avg_cost += bn_loss / train_total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 2.968802214
Epoch: 0002 cost = 2.965355873
Epoch: 0003 cost = 2.964182854
Epoch: 0004 cost = 2.967131376
Epoch: 0005 cost = 2.966554880
Epoch: 0006 cost = 2.961052895
Epoch: 0007 cost = 2.965565443
Epoch: 0008 cost = 2.963274956
Epoch: 0009 cost = 2.970722198
Epoch: 0010 cost = 2.964244127
Epoch: 0011 cost = 2.960146189
Epoch: 0012 cost = 2.968246460
Epoch: 0013 cost = 2.964675426
Epoch: 0014 cost = 2.961203098
Epoch: 0015 cost = 2.970076799
Learning finished


In [27]:
#test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것)
#X_test 불러올 때 view를 사용하여 차원 변환할 것/ Y_test를 불러올때 labels사용
#accuracy의 초기 값 설정(0으로) 꼭 할 것

with torch.no_grad():
    model2.eval()
    
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model2(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    bn_acc = correct_prediction.float().mean()
    print('Accuracy:', bn_acc.item())
    
    ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    
    print('Label: ', Y_single_data.item())
    single_prediction = model2(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.10220000147819519
Label:  8
Prediction:  8


In [28]:
#300개라면?

# Layer 쌓기 (조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), ReLU 함수 사용, Batch normalization하기)
# 각 Layer의 Hidden node 수 : 1st Layer (784,300), 2nd Layer(300,300),3rd Layer(300,10)

linear7 = torch.nn.Linear(784, 300, bias=True)
linear8 = torch.nn.Linear(300, 300, bias=True)
linear9 = torch.nn.Linear(300, 10, bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=0.3)

bn5 = torch.nn.BatchNorm1d(300)
bn6 = torch.nn.BatchNorm1d(300)

#xavier initialization을 이용하여 각 layer의 weight 초기화

torch.nn.init.xavier_uniform_(linear7.weight)
torch.nn.init.xavier_uniform_(linear8.weight)
torch.nn.init.xavier_uniform_(linear9.weight)


# torch.nn.Sequential을 이용하여 model 정의하기(쌓는 순서: linear-Batch Normalization Layer - ReLU- DropOut)

model3 = torch.nn.Sequential(linear7, bn5, relu, dropout,
                             linear8, bn6, relu, dropout,
                             linear9).to(device)

In [29]:
# Loss Function 정의하기 (CrossEntropy를 사용할 것!)

criterion = torch.nn.CrossEntropyLoss().to(device)

#optimizer 정의하기 (Adam optimizer를 사용할 것!)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#cost 계산을 위한 변수 설정
train_total_batch = len(train_loader)

#Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것) 
model3.train()
for epoch in range(training_epochs):
    avg_cost = 0
    
    #train dataset을 불러오고(X,Y 불러오기), back propagation과 optimizer를 사용하여 loss를 최적화하는 코드
    for X, Y in train_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model3(X)
        bn_loss = criterion(hypothesis, Y)
        bn_loss.backward()
        optimizer.step()

        avg_cost += bn_loss / train_total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 3.058442593
Epoch: 0002 cost = 3.052350283
Epoch: 0003 cost = 3.057399035
Epoch: 0004 cost = 3.046210766
Epoch: 0005 cost = 3.052311659
Epoch: 0006 cost = 3.056437492
Epoch: 0007 cost = 3.054040670
Epoch: 0008 cost = 3.047434330
Epoch: 0009 cost = 3.057872057
Epoch: 0010 cost = 3.055835485
Epoch: 0011 cost = 3.058263302
Epoch: 0012 cost = 3.047095776
Epoch: 0013 cost = 3.053744555
Epoch: 0014 cost = 3.051045179
Epoch: 0015 cost = 3.055782318
Learning finished


In [30]:
#test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것)
#X_test 불러올 때 view를 사용하여 차원 변환할 것/ Y_test를 불러올때 labels사용
#accuracy의 초기 값 설정(0으로) 꼭 할 것

with torch.no_grad():
    model3.eval()
    
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model3(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    bn_acc = correct_prediction.float().mean()
    print('Accuracy:', bn_acc.item())
       
    ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    
    print('Label: ', Y_single_data.item())
    single_prediction = model3(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.07660000026226044
Label:  2
Prediction:  6


In [31]:
#50개라면?

# Layer 쌓기 (조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), ReLU 함수 사용, Batch normalization하기)
# 각 Layer의 Hidden node 수 : 1st Layer (784,50), 2nd Layer(50,50),3rd Layer(50,10)

linear10 = torch.nn.Linear(784, 50, bias=True)
linear11 = torch.nn.Linear(50, 50, bias=True)
linear12 = torch.nn.Linear(50, 10, bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=0.3)

bn7 = torch.nn.BatchNorm1d(50)
bn8 = torch.nn.BatchNorm1d(50)

#xavier initialization을 이용하여 각 layer의 weight 초기화

torch.nn.init.xavier_uniform_(linear10.weight)
torch.nn.init.xavier_uniform_(linear11.weight)
torch.nn.init.xavier_uniform_(linear12.weight)


# torch.nn.Sequential을 이용하여 model 정의하기(쌓는 순서: linear-Batch Normalization Layer - ReLU- DropOut)

model4 = torch.nn.Sequential(linear10, bn7, relu, dropout,
                             linear11, bn8, relu, dropout,
                             linear12).to(device)

In [32]:
# Loss Function 정의하기 (CrossEntropy를 사용할 것!)

criterion = torch.nn.CrossEntropyLoss().to(device)

#optimizer 정의하기 (Adam optimizer를 사용할 것!)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#cost 계산을 위한 변수 설정
train_total_batch = len(train_loader)

#Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것) 
model4.train()
for epoch in range(training_epochs):
    avg_cost = 0
    
    #train dataset을 불러오고(X,Y 불러오기), back propagation과 optimizer를 사용하여 loss를 최적화하는 코드
    for X, Y in train_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model4(X)
        bn_loss = criterion(hypothesis, Y)
        bn_loss.backward()
        optimizer.step()

        avg_cost += bn_loss / train_total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 2.710007906
Epoch: 0002 cost = 2.709723949
Epoch: 0003 cost = 2.704931974
Epoch: 0004 cost = 2.713452101
Epoch: 0005 cost = 2.713769674
Epoch: 0006 cost = 2.710701942
Epoch: 0007 cost = 2.716084003
Epoch: 0008 cost = 2.710937977
Epoch: 0009 cost = 2.707369804
Epoch: 0010 cost = 2.711225271
Epoch: 0011 cost = 2.711918592
Epoch: 0012 cost = 2.716945410
Epoch: 0013 cost = 2.710141897
Epoch: 0014 cost = 2.717124939
Epoch: 0015 cost = 2.715279102
Learning finished


In [33]:
#test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것)
#X_test 불러올 때 view를 사용하여 차원 변환할 것/ Y_test를 불러올때 labels사용
#accuracy의 초기 값 설정(0으로) 꼭 할 것

with torch.no_grad():
    model4.eval()
    
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model4(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    bn_acc = correct_prediction.float().mean()
    print('Accuracy:', bn_acc.item())
       
    ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    
    print('Label: ', Y_single_data.item())
    single_prediction = model4(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.14740000665187836
Label:  9
Prediction:  7
